In [55]:
import pandas as pd  
import numpy as np
import math
# reading the data

Amk_data = pd.read_csv(r"C:\Users\Nathan\Desktop\Assignment 2\ado\AMK Resale flat prices with coords.csv")
carpark_data = pd.read_csv(r"C:\Users\Nathan\Desktop\Assignment 2\ado\AMK carpark-information.csv")
taxi_data = pd.read_csv(r"C:\Users\Nathan\Desktop\Assignment 2\ado\AMK taxi information.csv")

Amk_data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,Address,Latitude,Longitude
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44,Improved,1979,61 years 04 months,232000,406 ANG MO KIO AVE 10,1.362005,103.853880
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67,New Generation,1978,60 years 07 months,250000,108 ANG MO KIO AVE 4,1.370966,103.838202
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,262000,602 ANG MO KIO AVE 5,1.380709,103.835368
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68,New Generation,1980,62 years 01 month,265000,465 ANG MO KIO AVE 10,1.366201,103.857201
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,265000,601 ANG MO KIO AVE 5,1.381041,103.835132


In [56]:
carpark_data.head()

,car_park_no,address,x_coord,y_coord,Latitude,Longitude,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,car_park_decks,gantry_height,car_park_basement
0,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.44,39012.67,1.369085,103.834985,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,0,0.0,N
1,AK31,BLK 302/348 ANG MO KIO ST 31,29482.03,38684.18,1.366114,103.846636,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,0,0.0,N
2,AK52,BLOCK 513 ANG MO KIO ST 53,29889.35,39382.81,1.372432,103.850296,SURFACE CAR PARK,COUPON PARKING,WHOLE DAY,NO,YES,0,0.0,N
3,AK6,BLOCK 728 ANG MO KIO AVE 6,29283.73,39647.89,1.374829,103.844854,SURFACE CAR PARK,COUPON PARKING,7AM-10.30PM,SUN & PH FR 7AM-10.30PM,NO,0,0.0,N
4,AK83,BLK 5022 TO 5095 ANG MO KIO INDUSTRIAL PARK 2,31397.22,39851.62,1.376672,103.863845,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,0,0.0,N


In [57]:
taxi_data.head()

,Name,Long round,Lat round,geom,Town,Street
0,kml_114,103.829206,1.371916,"1.371916,103.829206",Ang Mo Kio,20 Jalan Leban
1,kml_257,103.844371,1.382114,"1.382114,103.844371",Ang Mo Kio,3000 Yio Chu Kang Bus Interchange
2,kml_258,103.849393,1.369924,"1.369924,103.849393",Ang Mo Kio,Ang Mo Kio Avenue 8


In [58]:
def haversine(lo1, la1, lo2, la2, d2r):
    dist = 0

    dlong = d2r * (lo2 - lo1)
    dlat = d2r * (la2 - la1)

    a = pow(math.sin(dlat/2),2) + math.cos(la1*d2r) * math.cos(la2*d2r) * pow(math.sin(dlong/2),2)
    c = 2 * math.atan2(math.sqrt(a),math.sqrt(1-a))
    dist = c * 6367

    return dist

In [64]:
d2r = math.pi/180
output = pd.DataFrame()

for index_ap, row_ap in Amk_data.iterrows():
    for index_am, row_am in taxi_data.iterrows():
        
        Lat1 = row_ap['Latitude']
        Lon1 = row_ap['Longitude']
        Lat2 = row_am['Lat round']
        Lon2 = row_am['Long round']
        
        dist = haversine(Lon1, Lat1, Lon2, Lat2, d2r)
        
        if dist < 2:
            new_row = {'house':row_ap['Address'],
                       'Block':row_ap['block'],
                       'Distance':dist,
                      'Name':row_am['Name']}
            new_row = pd.DataFrame(new_row, index=[len(output)])
            new_row= pd.merge(new_row, taxi_data.iloc[[index_am]])
            output = pd.concat([output, new_row])

In [65]:
d2r = math.pi/180
output2 = pd.DataFrame()

for index_ap, row_ap in Amk_data.iterrows():
    for index_am, row_am in carpark_data.iterrows():
        
        Lat1 = row_ap['Latitude']
        Lon1 = row_ap['Longitude']
        Lat2 = row_am['Latitude']
        Lon2 = row_am['Longitude']
        
        dist = haversine(Lon1, Lat1, Lon2, Lat2, d2r)
        
        if dist < 0.5:
            new_row2 = {'house':row_ap['Address'],
                       'Block':row_ap['block'],
                       'Distance':dist,
                      'car_park_no':row_am['car_park_no']}
            new_row2 = pd.DataFrame(new_row2, index=[len(output2)])
            new_row2= pd.merge(new_row2, carpark_data.iloc[[index_am]])
            output2 = pd.concat([output2, new_row2])

In [68]:
output.head(20)

,house,Block,Distance,Name,Long round,Lat round,geom,Town,Street
0,406 ANG MO KIO AVE 10,406,1.011414,kml_258,103.849393,1.369924,"1.369924,103.849393",Ang Mo Kio,Ang Mo Kio Avenue 8
0,108 ANG MO KIO AVE 4,108,1.004940,kml_114,103.829206,1.371916,"1.371916,103.829206",Ang Mo Kio,20 Jalan Leban
0,108 ANG MO KIO AVE 4,108,1.415726,kml_257,103.844371,1.382114,"1.382114,103.844371",Ang Mo Kio,3000 Yio Chu Kang Bus Interchange
0,108 ANG MO KIO AVE 4,108,1.248641,kml_258,103.849393,1.369924,"1.369924,103.849393",Ang Mo Kio,Ang Mo Kio Avenue 8
0,602 ANG MO KIO AVE 5,602,1.193055,kml_114,103.829206,1.371916,"1.371916,103.829206",Ang Mo Kio,20 Jalan Leban
0,602 ANG MO KIO AVE 5,602,1.012263,kml_257,103.844371,1.382114,"1.382114,103.844371",Ang Mo Kio,3000 Yio Chu Kang Bus Interchange
0,602 ANG MO KIO AVE 5,602,1.965672,kml_258,103.849393,1.369924,"1.369924,103.849393",Ang Mo Kio,Ang Mo Kio Avenue 8
0,465 ANG MO KIO AVE 10,465,0.961027,kml_258,103.849393,1.369924,"1.369924,103.849393",Ang Mo Kio,Ang Mo Kio Avenue 8
0,601 ANG MO KIO AVE 5,601,1.208997,kml_114,103.829206,1.371916,"1.371916,103.829206",Ang Mo Kio,20 Jalan Leban
0,601 ANG MO KIO AVE 5,601,1.033318,kml_257,103.844371,1.382114,"1.382114,103.844371",Ang Mo Kio,3000 Yio Chu Kang Bus Interchange


In [69]:
output.shape

(13240, 9)

In [70]:
output2.head(20)

,house,Block,Distance,car_park_no,address,x_coord,y_coord,Latitude,Longitude,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,car_park_decks,gantry_height,car_park_basement
0,406 ANG MO KIO AVE 10,406,0.399005,AM96,BLK 352A ANG MO KIO STREET 32,29961.10,38457.91,1.364068,103.850940,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,12,2.15,N
0,406 ANG MO KIO AVE 10,406,0.264036,A25,BLK 330/337 ANG MO KIO AVE 8,30037.57,38313.63,1.362763,103.851628,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,4.50,N
0,406 ANG MO KIO AVE 10,406,0.230828,A38,BLK 407/410/421 ANG MO KIO AVE 10,30367.08,38445.70,1.363957,103.854588,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,4.50,N
0,406 ANG MO KIO AVE 10,406,0.097921,A39,BLK 401/405 ANG MO KIO AVE 10,30309.63,38134.69,1.361145,103.854072,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,0.00,N
0,406 ANG MO KIO AVE 10,406,0.346763,A40,BLK 471/476 ANG MO KIO ST 44,30633.47,38267.95,1.362350,103.856982,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,9.99,N
0,406 ANG MO KIO AVE 10,406,0.469822,A41,BLK 466/470 ANG MO KIO ST 44,30627.96,38553.32,1.364930,103.856933,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,0.00,N
0,406 ANG MO KIO AVE 10,406,0.293688,A86,BLK 4001-4003/4026-4028/4033-4035 ANG MO KIO I...,30582.10,38215.84,1.361878,103.856521,SURFACE CAR PARK,COUPON PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,0.00,N
0,108 ANG MO KIO AVE 4,108,0.414058,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.44,39012.67,1.369085,103.834985,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,0,0.00,N
0,108 ANG MO KIO AVE 4,108,0.340428,AM14,BLK 227 ANG MO KIO ST 23,28777.07,38973.95,1.368735,103.840301,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES,6,2.15,N
0,108 ANG MO KIO AVE 4,108,0.284466,AM16,BLK 256A ANG MO KIO STREET 21,28267.06,39151.83,1.370343,103.835718,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,2,2.15,N


In [71]:
output2.shape

(80915, 17)

In [72]:
output.to_csv('Amk to taxi.csv', index=False)

In [73]:
output2.to_csv('Amk to carpark.csv', index=False)

In [74]:
info_data = pd.read_csv(r"C:\Users\Nathan\Desktop\Assignment 2\ado\AMK property information.csv")

In [75]:
info_data.head()

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,...,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,Address,Latitude,Longitude
0,101,ANG MO KIO AVE 3,12,1977,Y,N,N,N,N,N,...,0,0,0,0,0,0,0,101 ANG MO KIO AVE 3,1.370262,103.839468
1,102,ANG MO KIO AVE 3,12,1977,Y,N,N,Y,N,N,...,0,0,0,0,0,0,0,102 ANG MO KIO AVE 3,1.370650,103.839410
2,103,ANG MO KIO AVE 3,12,1977,Y,N,N,Y,N,N,...,0,0,0,0,0,0,0,103 ANG MO KIO AVE 3,1.371017,103.839305
3,104A,ANG MO KIO ST 11,12,1994,Y,N,N,N,N,N,...,60,0,0,0,0,0,0,104A ANG MO KIO ST 11,1.371827,103.838804
4,104B,ANG MO KIO ST 11,12,1994,Y,N,N,N,N,N,...,0,0,0,0,0,0,0,104B ANG MO KIO ST 11,1.371512,103.838459


In [76]:
Amk_data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,Address,Latitude,Longitude
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44,Improved,1979,61 years 04 months,232000,406 ANG MO KIO AVE 10,1.362005,103.853880
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67,New Generation,1978,60 years 07 months,250000,108 ANG MO KIO AVE 4,1.370966,103.838202
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,262000,602 ANG MO KIO AVE 5,1.380709,103.835368
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68,New Generation,1980,62 years 01 month,265000,465 ANG MO KIO AVE 10,1.366201,103.857201
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,265000,601 ANG MO KIO AVE 5,1.381041,103.835132


In [79]:
merged_data = pd.DataFrame()

merged_data = pd.merge(info_data, Amk_data)

In [80]:
merged_data.head()

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,...,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,101,ANG MO KIO AVE 3,12,1977,Y,N,N,N,N,N,...,ANG MO KIO,5 ROOM,101,ANG MO KIO AVE 3,01 TO 03,117,Standard,1978,60 years 01 month,500000
1,101,ANG MO KIO AVE 3,12,1977,Y,N,N,N,N,N,...,ANG MO KIO,5 ROOM,101,ANG MO KIO AVE 3,04 TO 06,117,Standard,1978,59 years,552000
2,101,ANG MO KIO AVE 3,12,1977,Y,N,N,N,N,N,...,ANG MO KIO,5 ROOM,101,ANG MO KIO AVE 3,04 TO 06,117,Standard,1978,58 years 03 months,528000
3,101,ANG MO KIO AVE 3,12,1977,Y,N,N,N,N,N,...,ANG MO KIO,5 ROOM,101,ANG MO KIO AVE 3,10 TO 12,117,Standard,1978,58 years 02 months,564000
4,101,ANG MO KIO AVE 3,12,1977,Y,N,N,N,N,N,...,ANG MO KIO,5 ROOM,101,ANG MO KIO AVE 3,04 TO 06,117,Standard,1978,57 years 05 months,506888


In [81]:
merged_data.to_csv('AMK combined.csv', index=False)